In [4]:
import praw
from time import sleep

# Configura tu conexión con Reddit
reddit = praw.Reddit(
    client_id="XEVqTpGA9HE6MNujQAWt1w",
    client_secret="bv4J3usOH-V__I4tGD__bLoWnGA_Sg",
    user_agent="Testing_api",
    read_only=True
)

# Define el subreddit y la búsqueda
subreddit_name = "flatearth"

queries_list = [
    "flat earth",
    #"flat earth theory",
    #"flat earth society",
    #"flatearth",
    #"flatearth theory",
    #"flatearth society",
    #"flat earth news",
    #"flat earth memes",
    #"flat earth videos",
    #"flat earth documentary",
    #"flat earth",
    #"flat",
    #"globe",
    #"round earth",
    #"NASA"
]

resultados_totales = set()

subredit = reddit.subreddit(subreddit_name)

for query in queries_list:
    print(f"Buscando en el subreddit {subreddit_name} con la consulta '{query}'")
    resultados = subredit.search(query, limit=100)
    
    for submission in resultados:
        # Verifica si el título ya está en los resultados totales
        if submission.title not in resultados_totales:
            resultados_totales.add(submission)

Buscando en el subreddit flatearth con la consulta 'flat earth'


In [5]:
post = list(resultados_totales)[0]

In [6]:
# Extraer el título y el contenido del post asi como el nombre del autor

print(f"Title: {post.title}")
print(f"Author: {post.author}")
print(f"Content: {post.selftext}")
print(f"URL: {post.url}")
print(f"Upvotes: {post.ups}")
print(f"Downvotes: {post.downs}")
print(f"Comments: {post.num_comments}")
print(f"Created: {post.created}")
print(f"Subreddit: {post.subreddit}")
print(f"Post ID: {post.id}")
print(f"Post URL: {post.url}")
print(f"Post Score: {post.score}")

#Navegar el arbol de comentarios
comentarios = post.comments

for comentario in comentarios:
    print(f"Comentario: {comentario.body}")
    print(f"Autor: {comentario.author}")
    print(f"Upvotes: {comentario.ups}")
    print(f"Downvotes: {comentario.downs}")
    print(f"Created: {comentario.created}")
    print(f"Comentario ID: {comentario.id}")
    print(f"Comentario URL: {comentario.permalink}")
    print(f"Comentario Score: {comentario.score}")
    

    

Title: What it’s like arguing with a flat earth enthusiast…. 😂
Author: EasternCranberry559
Content: 
URL: https://i.redd.it/1cfpcyimxvwa1.jpg
Upvotes: 896
Downvotes: 0
Comments: 33
Created: 1732650846.0
Subreddit: flatearth
Post ID: 1h0l1qn
Post URL: https://i.redd.it/1cfpcyimxvwa1.jpg
Post Score: 896
Comentario: Actually, for them its like shooting your chess opponent in the head, then claiming you win because they can't make any moves.

They ban you if you argue against them
Autor: Eric_Prozzy
Upvotes: 31
Downvotes: 0
Created: 1732651617.0
Comentario ID: lz4nh73
Comentario URL: /r/flatearth/comments/1h0l1qn/what_its_like_arguing_with_a_flat_earth_enthusiast/lz4nh73/
Comentario Score: 31
Comentario: We quite literally can't play the board either because they ban us for trying so they are the pigeon here.
Autor: KamikazeTank
Upvotes: 42
Downvotes: 0
Created: 1732651326.0
Comentario ID: lz4mj6m
Comentario URL: /r/flatearth/comments/1h0l1qn/what_its_like_arguing_with_a_flat_earth_enthusi

In [7]:
!pip install tqdm

In [8]:
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

In [9]:
grafo = nx.Graph()

In [10]:
def get_results(subreddit, query_list, n_post = 10):
    resultados_totales = set()
    
    subredit = reddit.subreddit(subreddit)
    
    for query in query_list:
        print(f"Buscando en el subreddit {subreddit} con la consulta '{query}'")
        
        try:
        
            resultados = subredit.search(query, limit=n_post)
            
            for submission in resultados:
                # Verifica si el título ya está en los resultados totales
                if submission.title not in resultados_totales:
                    resultados_totales.add(submission)
                    
        except Exception:
            continue
                
    return resultados_totales

In [11]:
def scrapear_subreddit(subreddit, query_list, n_post = 10, max_comments = None):
    
    # Obtener los resultados
    resultados_totales = get_results(subreddit, query_list, n_post)

    # Agregar nodo raíz al grafo
    grafo.add_node(subreddit)
    
    # Agregar los nodos y aristas al grafo
    for post in tqdm(list(resultados_totales)):
        if post.author:
            grafo.add_node(post.author.name,subreddit=subreddit)
            grafo.add_edge(subreddit, post.author.name)
            
        while True:
            try:
                post.comments.replace_more(limit=None)
                break
            except Exception:
                print('demasiadas peticiones')
                sleep(10)
            
        comentarios = sorted(post.comments.list(), key=lambda x: x.score, reverse=True)
        
        for comentario in comentarios:
            if comentario.author:
                #comprueba que el nodo ya existe
                if grafo.has_node(comentario.author.name):
                    if not grafo.has_edge(post.author.name, comentario.author.name):
                        grafo.add_edge(post.author.name, comentario.author.name)
                        
        if max_comments is not None:
            comentarios = comentarios[:max_comments]
        
        for comentario in comentarios:
            if comentario.author:
                grafo.add_node(comentario.author.name, subreddit=subreddit)
                #comprobar que no exista ya la arista
                if not grafo.has_edge(post.author.name, comentario.author.name):
                    grafo.add_edge(post.author.name, comentario.author.name)


In [141]:
scrapear_subreddit('flatearth', ['flatearth'], n_post = 10)
scrapear_subreddit('antivaccine', ['antivaccine'], n_post = 10)


Buscando en el subreddit flatearth con la consulta 'flatearth'


 10%|█         | 1/10 [00:03<00:29,  3.32s/it]


KeyboardInterrupt: 

In [95]:
#dame las estadisticas del grafo
nx.number_of_nodes(grafo)

1555

In [135]:
#exporta el grafo a un archivo GML

nx.write_gml(grafo, "grafo.gml")

In [14]:
lista_comunes = ["critical thinking", "question everything", "pseudoscience",  "debunk", "denial", "conspiracy"]

subreddit_keywords = {
    'FlatEarth': [
        'flat earth theory',
        'ice wall discovery',
        'antarctic cover‑up',
        'horizon anomaly',
        'ground is level',
        *lista_comunes
    ],
    'moonlandinghoax': [
        'fake lunar mission',
        'studio filmed moon',
        'camera glitch proof',
        'lunar module lies',
        'NASA backlot conspiracy',
        *lista_comunes
    ],
    'aliens': [
        'ufo disclosure',
        'area 51 secrets',
        'extraterrestrial visit',
        'crop circle proof',
        'alien cover‑up',
        *lista_comunes
    ],
    'No5G': [
        '5g health risks',
        'radiation cover‑up',
        'cell tower danger',
        'electromagnetic fraud',
        'hidden toxicity',
        *lista_comunes
    ],
    'chemtrails': [
        'chemical spraying',
        'aerosol cover‑up',
        'geoengineering proof',
        'toxics in sky',
        'secret air program',
        *lista_comunes
    ],
    'antiVaxx': [
        'big pharma lies',
        'vaccine injury cover‑up',
        'unreported side effects',
        'injection fraud',
        'medical experimentation',
        *lista_comunes
    ],
    'pesticideconspiracy': [
        'pesticide runoff cover‑up',
        'agrochemical conspiracy',
        'glyphosate danger',
        'farmers whistleblower',
        'toxic crop spraying',
        *lista_comunes
    ],
    'CovidConspiracy': [
        'plandemic proof',
        'virus lab leak cover‑up',
        'mask mandate fraud',
        'PCR gate',
        'vaccine passport tyranny',
        *lista_comunes
    ],
    'climatechangeisalie': [
        'co2 myth',
        'green agenda',
        'warming scam',
        'carbon tax cover‑up',
        'environment fraud',
        *lista_comunes
    ],
    'newworldorder': [
        'one world government',
        'global elite plan',
        'shadow government proof',
        'population control agenda',
        'nwo conspiracy',
        *lista_comunes
    ],
    
    'SimulationTheory': [
        'we live in a simulation',
        'matrix is real',
        'simulated reality proof',
        'glitch in the matrix',
        'reality is fake',
        'computer-generated universe',
        'consciousness and simulation',
        'base reality hypothesis',
        'elon musk simulation',
        'philosophical zombies',
        *lista_comunes
    ],
    'Plandemic': [
        'covid was planned',
        'virus patent evidence',
        'event 201 proof',
        'pandemic simulation cover-up',
        'vaccine agenda',
        'plandemic documentary',
        'mass control operation',
        'false positive PCR',
        'big pharma profits',
        'government fear propaganda',
        *lista_comunes
    ],
    'DebateHolocaust': [
        'holocaust exaggeration claims',
        'gas chamber doubts',
        'revisionist history',
        'ww2 narrative question',
        'auschwitz controversy',
        'truth about holocaust',
        'holocaust denial arguments',
        'zionist agenda claims',
        'nuremberg trials critique',
        'forbidden historical facts',
        *lista_comunes
    ],
    'Epstein': [
        'epstein didn’t kill himself',
        'epstein client list',
        'ghislaine maxwell secrets',
        'elite pedophile ring',
        'epstein island evidence',
        'blackmail operation',
        'intelligence agency ties',
        'jeffrey epstein cover‑up',
        'flight logs secrets',
        'missing surveillance tapes',
        *lista_comunes
    ]
}

In [15]:
#crea un grafo vacio
grafo = nx.Graph()

for subreddit, queries in subreddit_keywords.items():
    scrapear_subreddit(subreddit, queries, n_post = 10)

# Exporta el grafo a un archivo GML
nx.write_gml(grafo, "grafo.gml")

Buscando en el subreddit FlatEarth con la consulta 'flat earth theory'
Buscando en el subreddit FlatEarth con la consulta 'ice wall discovery'
Buscando en el subreddit FlatEarth con la consulta 'antarctic cover‑up'
Buscando en el subreddit FlatEarth con la consulta 'horizon anomaly'
Buscando en el subreddit FlatEarth con la consulta 'ground is level'
Buscando en el subreddit FlatEarth con la consulta 'critical thinking'
Buscando en el subreddit FlatEarth con la consulta 'question everything'
Buscando en el subreddit FlatEarth con la consulta 'pseudoscience'
Buscando en el subreddit FlatEarth con la consulta 'debunk'
Buscando en el subreddit FlatEarth con la consulta 'denial'
Buscando en el subreddit FlatEarth con la consulta 'conspiracy'


100%|██████████| 102/102 [04:56<00:00,  2.91s/it]


Buscando en el subreddit moonlandinghoax con la consulta 'fake lunar mission'
Buscando en el subreddit moonlandinghoax con la consulta 'studio filmed moon'
Buscando en el subreddit moonlandinghoax con la consulta 'camera glitch proof'
Buscando en el subreddit moonlandinghoax con la consulta 'lunar module lies'
Buscando en el subreddit moonlandinghoax con la consulta 'NASA backlot conspiracy'
Buscando en el subreddit moonlandinghoax con la consulta 'critical thinking'
Buscando en el subreddit moonlandinghoax con la consulta 'question everything'
Buscando en el subreddit moonlandinghoax con la consulta 'pseudoscience'
Buscando en el subreddit moonlandinghoax con la consulta 'debunk'
Buscando en el subreddit moonlandinghoax con la consulta 'denial'
Buscando en el subreddit moonlandinghoax con la consulta 'conspiracy'


100%|██████████| 21/21 [00:13<00:00,  1.52it/s]


Buscando en el subreddit aliens con la consulta 'ufo disclosure'
Buscando en el subreddit aliens con la consulta 'area 51 secrets'
Buscando en el subreddit aliens con la consulta 'extraterrestrial visit'
Buscando en el subreddit aliens con la consulta 'crop circle proof'
Buscando en el subreddit aliens con la consulta 'alien cover‑up'
Buscando en el subreddit aliens con la consulta 'critical thinking'
Buscando en el subreddit aliens con la consulta 'question everything'
Buscando en el subreddit aliens con la consulta 'pseudoscience'
Buscando en el subreddit aliens con la consulta 'debunk'
Buscando en el subreddit aliens con la consulta 'denial'
Buscando en el subreddit aliens con la consulta 'conspiracy'


...

0it [00:00, ?it/s]


Buscando en el subreddit DebateHolocaust con la consulta 'holocaust exaggeration claims'
Buscando en el subreddit DebateHolocaust con la consulta 'gas chamber doubts'
Buscando en el subreddit DebateHolocaust con la consulta 'revisionist history'
Buscando en el subreddit DebateHolocaust con la consulta 'ww2 narrative question'
Buscando en el subreddit DebateHolocaust con la consulta 'auschwitz controversy'
Buscando en el subreddit DebateHolocaust con la consulta 'truth about holocaust'
Buscando en el subreddit DebateHolocaust con la consulta 'holocaust denial arguments'
Buscando en el subreddit DebateHolocaust con la consulta 'zionist agenda claims'
Buscando en el subreddit DebateHolocaust con la consulta 'nuremberg trials critique'
Buscando en el subreddit DebateHolocaust con la consulta 'forbidden historical facts'
Buscando en el subreddit DebateHolocaust con la consulta 'critical thinking'
Buscando en el subreddit DebateHolocaust con la consulta 'question everything'
Buscando en el s

100%|██████████| 10/10 [00:05<00:00,  1.79it/s]


Buscando en el subreddit Epstein con la consulta 'epstein didn’t kill himself'
Buscando en el subreddit Epstein con la consulta 'epstein client list'
Buscando en el subreddit Epstein con la consulta 'ghislaine maxwell secrets'
Buscando en el subreddit Epstein con la consulta 'elite pedophile ring'
Buscando en el subreddit Epstein con la consulta 'epstein island evidence'
Buscando en el subreddit Epstein con la consulta 'blackmail operation'
Buscando en el subreddit Epstein con la consulta 'intelligence agency ties'
Buscando en el subreddit Epstein con la consulta 'jeffrey epstein cover‑up'
Buscando en el subreddit Epstein con la consulta 'flight logs secrets'
Buscando en el subreddit Epstein con la consulta 'missing surveillance tapes'
Buscando en el subreddit Epstein con la consulta 'critical thinking'
Buscando en el subreddit Epstein con la consulta 'question everything'
Buscando en el subreddit Epstein con la consulta 'pseudoscience'
Buscando en el subreddit Epstein con la consulta 

100%|██████████| 134/134 [08:05<00:00,  3.62s/it] 


dañino:

tiera plana
antivacunas
moonlanding

no dañino
